<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [80]:
df = pd.read_csv("2024.csv")

In [81]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  304 non-null    object
 1   AP        304 non-null    object
 2   N01       304 non-null    int64 
 3   N02       304 non-null    int64 
 4   N03       304 non-null    int64 
 5   N04       304 non-null    int64 
 6   N05       304 non-null    int64 
 7   N06       304 non-null    int64 
 8   N07       304 non-null    int64 
 9   N08       304 non-null    int64 
 10  N09       304 non-null    int64 
 11  N10       304 non-null    int64 
 12  N11       304 non-null    int64 
 13  N12       304 non-null    int64 
 14  N13       304 non-null    int64 
 15  N14       304 non-null    int64 
 16  N15       304 non-null    int64 
 17  N16       304 non-null    int64 
 18  N17       304 non-null    int64 
 19  N18       304 non-null    int64 
 20  N19       304 non-null    int64 
 21  N20       304 no

In [82]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [83]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 5
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [84]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [85]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 32
ltsm_units = 128
dropout = 0.2

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(dropout))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(dropout))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(dropout))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [86]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.90, random_state=90)

# Train the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=200, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/200
1/1 - 15s - loss: 0.9947 - accuracy: 0.0345 - 15s/epoch - 15s/step
Epoch 2/200
1/1 - 0s - loss: 0.9828 - accuracy: 0.2069 - 19ms/epoch - 19ms/step
Epoch 3/200
1/1 - 0s - loss: 0.9725 - accuracy: 0.1034 - 16ms/epoch - 16ms/step
Epoch 4/200
1/1 - 0s - loss: 0.9594 - accuracy: 0.1379 - 19ms/epoch - 19ms/step
Epoch 5/200
1/1 - 0s - loss: 0.9414 - accuracy: 0.1724 - 20ms/epoch - 20ms/step
Epoch 6/200
1/1 - 0s - loss: 0.9133 - accuracy: 0.1379 - 16ms/epoch - 16ms/step
Epoch 7/200
1/1 - 0s - loss: 0.8896 - accuracy: 0.1379 - 16ms/epoch - 16ms/step
Epoch 8/200
1/1 - 0s - loss: 0.8659 - accuracy: 0.1379 - 18ms/epoch - 18ms/step
Epoch 9/200
1/1 - 0s - loss: 0.8447 - accuracy: 0.1379 - 16ms/epoch - 16ms/step
Epoch 10/200
1/1 - 0s - loss: 0.8199 - accuracy: 0.1724 - 16ms/epoch - 16ms/step
Epoch 11/200
1/1 - 0s - loss: 0.7598 - accuracy: 0.2069 - 16ms/epoch - 16ms/step
Epoch 12/200
1/1 - 0s - loss: 0.7101 - accuracy: 0.2414 - 17ms/epoch - 17ms/step
Epoch 13/200
1/1 - 0s - loss: 0.6676 -

In [87]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

10/10 [==============================] - 2s 6ms/step - loss: 1.4638 - accuracy: 0.1070
Test Loss: 1.4637984037399292, Test Accuracy: 0.10702341049909592
9/9 [==============================] - 0s 6ms/step - loss: 1.6203 - accuracy: 0.0333
Test Loss: 1.6203222274780273, Test Accuracy: 0.03333333507180214


In [88]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [97]:
import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'{label}.keras')
print("saved model")

saved model


In [90]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-90-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
299,2,5,6,9,18,19,21,25,26,30,31,33,34,44,48,50,60,65,67,69
300,8,11,17,18,24,26,30,31,33,38,39,44,45,46,48,51,54,63,64,65
301,1,3,7,9,10,12,14,15,19,22,25,26,31,35,36,45,51,54,56,65
302,4,5,8,20,21,22,24,29,34,36,37,39,40,41,43,45,46,52,53,55


In [91]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_10_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_10_numbers.append(number)

print("The top 10 indices:", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("The winning indices:", winning_indices)
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 4s 4s/step
The top 5 indices: [4 1 2 3 5]
The top 5 numbers: [15, 5, 12, 13, 16]


The top 10 indices: [ 4  1  2  3  5  0  6  7  8 19]
The top 10 numbers: [15, 5, 12, 13, 16, 2, 18, 20, 23, 65]


The predicted numbers for the lottery game are:   [ 2  7 10 14 19 20 23 25 27 30 33 36 37 39 43 47 49 55 59 65]
The actual numbers in the last lottery game were: [ 2  5 12 13 15 16 18 20 23 26 28 30 37 43 48 51 53 56 59 65]


The winning indices: [ 0  7  8 11 12 13 18 19]
winners: [ 2 20 23 30 37 43 59 65]
0.4


# Predict the next set of numbers

In [92]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game: [ 2  7 10 14 19 20 23 25 27 30 33 36 37 39 43 47 49 55 59 65]


The top 5 indices [4 3 0 1 2]
The top 5 numbers: [18, 13, 3, 6, 9]


The top 10 indices [ 4  3  0  1  2  7  5  6 19  8]
The top 10 numbers: [18, 13, 3, 6, 9, 25, 18, 21, 65, 26]


winning indices [ 0  7  8 11 12 13 18 19]
winning numbers [3, 25, 26, 34, 36, 36, 57, 65]


In [93]:
next = df.copy()
next = next.tail(10)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 21ms/step
The top 5 indices [0 2 1 7 9]
The top 5 numbers: [8, 12, 4, 23, 28]


The top 10 indices [ 0  2  1  7  9  4  8  5 10  3]
The top 10 numbers: [8, 12, 4, 23, 28, 11, 23, 13, 28, 5]


winning indices [ 0  7  8 11 12 13 18 19]
winning numbers [8, 23, 23, 29, 34, 33, 42, 61]


The predicted numbers: [[ 8  4 12  5 11 13 10 23 23 28 28 29 34 33 35 40 44 44 42 61]]
